In [1]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
from os import getcwd
import numpy as np 
MAX_NB_WORDS=40000 #defines the size of our vocabulary
MAX_SEQUENCE_LENGTH=100
VALIDATION_SPLIT=0.2 #what percent of training examples to be saved for testing
EMBEDDING_DIM=50

## Setting up the Training Data

We need all of our labels to be a simple 1 (offensive) or 0 (not offensive). Given that our dataset has a text label for hate speech and "offensive but not hate" speech, we need to map these text labels to integer ones using the Pandas library. #pandasIsYourDataScienceBestFriend


In [2]:
df = pd.read_csv(getcwd() + '/data/twitter-hate-speech.csv', encoding = "ISO-8859-1")
texts = df[:]['tweet_text']
labels = df[:]['does_this_tweet_contain_hate_speech']
labels = labels.map({'The tweet is not offensive': 0, 
                     'The tweet uses offensive language but not hate speech': 1, 
                     'The tweet contains hate speech': 1})
print("Have", len(labels), "training tweets")

Have 14509 training tweets


## Creating our Tokenizer

A tokenizer will turn each text into a sequence of Integers, with each integer being the index of a token in our dictionary. 

This tokenizer dictionary becomes our vocaulary and we need to cap the length (`MAX_NB_WORDS`) of it as english speakers have an uncanny ability to make up words of low frequency. We ignore these words for reasons of computational limits.

We use `fit_on_text` to make a dictionary from the training texts, while the `texts_to_sequences` transforms each work in the text and replaces it with the corresponding integer value.

We also pad our sequences to make every text sequence the same length. We cap these at 100, which is pretty high given the length of a tweet, but the neural net architecture I "borrowed" below was build for something much larger, so we're doing some padding.

Even though we're looking for a 0 or 1 value for a label, we still use `to_categorial` to give the cathe categories of "offensive" and "not offensive" with 0 or 1 values for each.

Note that we're saving a `word_index`, we'll need that later when we create our embedding layer.

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras import backend as K
K.set_image_dim_ordering('tf')

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

/Users/imack/.pyenv/versions/3.5.4/lib/python3.5/site-packages/keras_preprocessing/text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 28821 unique tokens.
Shape of data tensor: (14509, 100)
Shape of label tensor: (14509, 2, 2)


## Create the Pickle File

For sanity sakes, we output a text, the sequence which shows the word indexes, and the padded sequence to demostrate. 

From here, we want to dump our dictionary for later use. It becomes part of our model, as we will need it to encode new text we want to perform predictions on. 

In [4]:
import pickle

print(texts[125])
print(sequences[125])
print(data[125])

with open('model/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

@OfficialScarbs @JPizzleFIFA @ItzEmmo omg i said him and charlie look alike
[9586, 9587, 9588, 586, 4, 141, 70, 9, 4384, 105, 495]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0 9586 9587 9588
  586    4  141   70    9 4384  105  495]


## Train/Test Sets

We actually want to split our data into `x_train` (training texts), `y_train` (training labels), `x_test` (test texts), `y_test` (test labels). 

The training examples will be used by our algorithm to train the neural network, but we need to test at every step of the way and make sure we aren't "overfitting".

> *Overfitting "the production of an analysis that corresponds too closely or exactly to a particular set of data, and may therefore fail to fit additional data or predict future observations reliably" -[Oxford Dictionaries](https://www.lexico.com/en/definition/overfitting)*

In [5]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]


## Fits Like a GloVe

Now that we have our vectors, we need to load our pre-trained GloVe word vectors for turning our words into high-demensional vectors for understanding english words. We load the 50-dimensional dataset into a dictionary called `embeddings_index`. Which maps words to arrays of 50 integers. (this takes a minute to load the data)

[GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)

In [6]:
embeddings_index = {}
f = open('data/glove.twitter.27B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
print(embeddings_index['school'])

Found 1193514 word vectors.
[ 1.6875    0.47791   0.39499  -0.83214  -0.63072   0.23882   1.48
 -0.21343  -0.28804  -0.20694  -0.16641   0.37023  -4.6847    0.71942
 -0.35867  -0.15242  -0.70912  -0.097531 -1.4153    0.39974   0.11364
  0.58625  -0.094106  0.15768   0.89874  -0.06887   0.17851  -1.0488
 -0.28724  -0.75495  -0.075509 -0.54444   0.19466  -0.96681   0.15595
 -0.99427  -0.75542   1.2644    0.86891  -0.41762  -0.90746   0.84768
 -0.35308  -0.22458   0.34505  -0.37169   0.55209   0.50678  -0.4307
  0.92539 ]


## Embedding Layer

The Embedding Layer is the first layer of our "deep learning" network. It take positive integer indexes and turns them into dense vectors based on our GloVe dictionary. Using the `word_index` from our Tokenizer, this is the last step to get our data ready before training. 

In [7]:
from keras.layers import Embedding

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

## Training Our Model

The actual architecture of our model is an Embedding Layer, a Convolutional 1D layer, a polling layer, a n1D Convolutional layer, a pooling layer, a 1D Convlutional layer, a final pooling layer, and a fully connected layer (Dense) connected to the 2 outputs.

This is the real "building" of the model, but honesly I just picked it up from some example on the internet and remove some layers because our tweet window size is so small. We can play with the size of each layer (hyperparameter tuning) to get better results but this can take a lot of experimentation.

We run batches of 128 text examples each iteration for speed, iterating over all examples 5 times (number of epochs). This whole process will set the weights on our neural network.

Each iteraction we can see the accuracy increase until it starts to level off. A common trend with training networks. This model will train really quickly on a CPU, but for large datasets that involved much larger things like images, you'll definitely want to run on a GPU.

In [9]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

print(model.summary())

Training model.
(?, 50, 50)


Train on 11608 samples, validate on 2901 samples
Epoch 1/10


  128/11608 [..............................] - ETA: 24s - loss: 0.6966 - acc: 0.5078

  512/11608 [>.............................] - ETA: 6s - loss: 0.9309 - acc: 0.5293 

  896/11608 [=>............................] - ETA: 4s - loss: 0.7987 - acc: 0.5815

...

10368/11608 [=========================>....] - ETA: 0s - loss: 0.0799 - acc: 0.9723

10752/11608 [==========================>...] - ETA: 0s - loss: 0.0792 - acc: 0.9727

11136/11608 [===========================>..] - ETA: 0s - loss: 0.0780 - acc: 0.9732

11608/11608 [==============================] - 2s 140us/step - loss: 0.0766 - acc: 0.9736 - val_loss: 0.3982 - val_acc: 0.8873


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            1441100   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 46, 128)           32128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 9, 128)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5, 128)            82048     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
__________

## Trained Model

We now have a trained model and can run predictions on it! Let's try it out with an example texts.

In [19]:
sequences = tokenizer.texts_to_sequences(['I love talking to you about machine learning'])
padded_input = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

predictions = model.predict(padded_input)
print("Message is", predictions[0][1], "offensive")

array([[6.7541207e-04, 9.9932456e-01]], dtype=float32)

## Export the Model

Our last step it to explort the model files structure as a json file and its weights as an h5 file. These, along with the pickler dictionary constitute our model.

In [20]:
model_json = model.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model/model.h5")
print("Saved model to disk")


Saved model to disk
